In [1]:
import pandas as pd
pdix = pd.IndexSlice
f = open("day_7_input.txt")
input_list = f.read().split("\n")

In [2]:
df = pd.DataFrame(input_list, columns=["raw"])
#Separate into parent and children
df['parent'] = df['raw'].str.split(' contain ').str[0].str.replace('bags', '').str.replace('bag', '').str.strip()
df['raw_child'] = df['raw'].str.split(' contain ').str[1].str.replace('bags', '').str.replace('bag', '').str.strip()

In [3]:
#Expand the children into the dataframe
children_list = [f"child_{i}" for i in range(1, 5)]
df[children_list] = df['raw_child'].str.split(", ", expand=True)

#Remove unnecessary columns and stack
df = df[['parent', *children_list]].set_index('parent').stack().to_frame()
df.columns = ["child"]

#Split the 'child' column into quantity and type using regex
df['child'] = df['child'].str.strip().str.strip('.')
regex = r"([0-9]+)"
regex_2 = r"([a-zA-Z\ ]+)"
#Extract the quantity, set those with N/A values as "0" to identify which are the bottom-level bags, and set as an integer
df['qty'] = df['child'].str.extract(regex)
df.loc[df['qty'].isna(), 'qty'] = 0
df['qty'] = df['qty'].astype(int)
#Extract type, clean up a bit, then remove unnecessary interim columns
df['type'] = df['child'].str.extract(regex_2).loc[:, 0].str.strip()
df = df[['type', 'qty']]

In [4]:
#Create an inverse of the dataframe that contains the parents of a child
inverted_df = df['type'].reset_index().drop(columns = 'level_1').groupby('type')['parent'].apply(list).to_frame()
inverted_df

,parent
type,
bright aqua,"[faded chartreuse, posh aqua, pale violet]"
bright beige,"[wavy green, faded black]"
bright black,"[bright chartreuse, wavy red, vibrant coral, d..."
bright blue,"[posh fuchsia, mirrored white, striped tomato,..."
bright bronze,"[faded chartreuse, wavy black, dark gray, clea..."
...,...
wavy purple,"[dull olive, wavy fuchsia, clear maroon]"
wavy silver,[striped gold]
wavy turquoise,"[striped magenta, pale violet]"


In [5]:
def day_7_part_1():
    #Create a queueueueue, or stack, that we will iterate through
    queueueueue = ["shiny gold"]
    valid_parents = set()
    
    #This will loop until the queueueueue is empty
    while len(queueueueue) > 0:
        #Remove the first item in the queueueue from the list
        subject = queueueueue.pop(0)
        #Add the parents to the set containing the valid parents. This is to remove redundancies automatically
        valid_parents.add(subject)
        #Try is here to catch top-level parents. If it throws a KeyError, there's no parents, and thus we are at the top level
        try:
            parents = inverted_df.loc[subject, 'parent']
        except KeyError:
            continue
        #Add the parents to the queue. NOTE: THIS WILL NOT REMOVE DUPLICATES. IMPORTANT TO STRESS
        queueueueue += parents
    
    #After we're out of the queue but before we return, we remove "Shiny gold" from the equation
    valid_parents.remove("shiny gold")
    return len(valid_parents)

In [6]:
day_7_part_1()

235

In [7]:
def day_7_part_2(subject = "shiny gold"):

    #This will retrieve a view of the dataframe that only has the appropriate children of the "subject" in question
    children_df = df.loc[pdix[subject, :], :]
    total_bags = 0

    #Iterate over the view retrieved above
    for [parent, idx], [child, qty] in children_df.iterrows():
        #If quantity is zero, we have a bottom-level bag and can return a 1
        if qty == 0:
            return 1
        
        #Otherwise, we will recurse with the child bag and multiply by the quantity of the bags within with the quantity of that specific bag held within the subject
        total_bags += qty * day_7_part_2(child)
    
    #Once outside the loop, we've completed how many bags are located inside. If the subject is "shiny gold", then we've reached the end and return the total bags. Otherwise, we return the amount of bags contained within plus one, to represent the subject itself.
    if subject == "shiny gold":
        return total_bags
    else:
        return total_bags + 1

In [8]:
day_7_part_2()

158493